In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Sentiment Analysis using Large Language Model

## Installation of modules used in this tutorial

In [ ]:
!pip install -U sentence-transformers
!pip install faiss-gpu
!pip install datasets
!pip install tabulate
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=51f4064fffdc32546e8761f51f23cb925b59c0ad9c0cc914077f01ac6d6e8fde
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, AutoModelForSeq2SeqLM
from collections import defaultdict
from sklearn.metrics import accuracy_score, classification_report
import faiss
import os
import random
from copy import deepcopy
random.seed(1234)

## Loading of the dataset

SST2, or the Stanford Sentiment Treebank version 2, is a widely-used dataset for sentiment analysis tasks, typically involving classifying text (such as movie reviews) into positive or negative categories. The total number of samples present in the dataset is 8741 of which 6920 training samples and 1821 test samples

In [ ]:
pd.set_option('display.max_rows', None) # To display all rows
pd.set_option('display.max_columns', None) # To display all columns
pd.set_option('display.width', 1000) # Adjust the width to fit your screen
pd.set_option('display.max_colwidth', None)

In [ ]:
dataset = load_dataset("gpt3mix/sst2")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
df_train = pd.DataFrame(dataset['train'])
print(df_train.head(10).to_markdown(index=False)) # 10 examples are demonstrated here

| text                                                                                                                                                                                                                                |   label |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|
| The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .                                               |       0 |
| The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth . |       0 |
| Singer\/composer Bryan Adams c

## Splitting training and testing



In [ ]:
def create_dataset_lable(d):
    lst = []
    text = d['text']
    label = d['label']
    for i, t in enumerate(tqdm(text)):
        lst.append((t,label[i]))
    return lst

In [ ]:
train_data = create_dataset_lable(dataset['train'])
test_data = create_dataset_lable(dataset['test'])

100%|██████████| 1821/1821 [00:00<00:00, 874293.45it/s]


#Learning in context is facilitated by a large language model, which incorporates few-shot demonstrations created using both random and dense retriever methods.

## Open Souce LLM FlanT5 with the help of Hugginface PipeLine

Flan-T5 stands for "Fine-tuned LAnguage Net T5." The primary advancement Flan-T5 introduces is in its pretraining approach. Instead of training solely on a traditional language modeling objective, Flan-T5 is pretrained on a mixture of datasets with prompts that encourage the model to learn a wide range of tasks. This approach is known as "instruction tuning," where the model is exposed to tasks described by instructions, allowing it to better generalize across tasks it wasn't explicitly trained on. Size of the model 783M


In [ ]:
MODEL_NAME = "google/flan-t5-large"

tokenizer_flan = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model_flan = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, device_map="auto" ## acceclerate
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Create Few-shots demonstrations

## Create sentence embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('sentence-transformers/stsb-bert-base',device="cuda")
def create_bert_embeddings(datasets):
    sentences = [d[0] for d in datasets]
    embeddings = model.encode(sentences,device="cuda",show_progress_bar=True,batch_size=128)
    return embeddings

.gitattributes:   0%|          | 0.00/744 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
train_sentences = create_bert_embeddings(train_data)
test_sentences = create_bert_embeddings(test_data)

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

## Indexing creation using faiss

Faiss is a library designed for efficient similarity search and clustering of dense vectors, supporting large-scale vector sets that may exceed RAM capacity. It offers comprehensive Python wrappers for both Python 2 and 3, while the core is written in C++, and includes GPU-accelerated algorithms for enhanced performance. Developed mainly by Meta AI Research, Faiss also benefits from contributions by external developers and includes tools for evaluation and parameter tuning.
<figure>
<center>
<img src='https://drive.google.com/uc?id=1Vwy-aXADx8Hvyiz7wJfMB-sTV_y_4J4O', width="200"/>
<figcaption>$K$ shot demonstration creation using KNN </figcaption></center>
</figure>


## Create Index using Faiss

In [ ]:
def create_index(embeddings,index_file):
    faiss.normalize_L2(embeddings)
    index = faiss.index_factory(768, "Flat", 0)
    index.train(embeddings)
    index.add(embeddings)
    faiss.write_index(index, os.path.join(index_file,"train_index.faiss"))

In [ ]:
create_index(train_sentences,"/content/.")

## Creation of the $k$ shots
For each test sample in the test dataset, we extract $k$ similar context-related sentences from the training dataset. To retrieve these similar sentences, we use a dense retriever like Faiss. Additionally, other types of retrieval models, such as TF-IDF, can also be utilized.


In [ ]:
## Helper Function to create few shots
def create_few_shots(test_emdb,train_data,test_data,index_file,top_k):
    top = dict()
    map_ = {1:"negative", 0: "positive"}
    index = faiss.read_index(os.path.join(index_file,"train_index.faiss"))
    faiss.normalize_L2(test_emdb)
    distance, index_l = index.search(test_emdb, top_k)
    index_list = index_l.tolist()
    # print(index_l.shape)
    for index,value in enumerate(tqdm(index_list)):
        for train_index in value:
            if test_data[index] in top:
                top[test_data[index]].append({"Input": train_data[train_index][0], "Output": map_[train_data[train_index][-1]]})
            else:
                top[test_data[index]] = []
                top[test_data[index]].append({"Input": train_data[train_index][0], "Output": map_[train_data[train_index][-1]]})
    return top,distance


def create_random_few_shots(train_data,test_data,top_k):
  top = dict()
  t = 0
  map_ = {1:"negative", 0: "positive"}
  train_cp = deepcopy(train_data)
  random.shuffle(train_cp)
  for index, value in enumerate(tqdm(test_data)):
    for train in train_cp:
      if value[0] in top:
        top[value[0]].append({"Input": train[0], "Output": map_[train[-1]]})
      else:
        top[value[0]] = []
        top[value[0]].append({"Input": train[0], "Output": map_[train[-1]]})
      if len(top[value[0]]) == top_k:
        break
  return top

In [ ]:
top_10,d = create_few_shots(test_sentences,train_data,test_data,"/content/.",10) ## Here Value of k is 10
top_10_random= create_random_few_shots(train_data,test_data,10) ## Here Value of k is 10

100%|██████████| 1821/1821 [00:00<00:00, 85144.78it/s]


In [ ]:
# Helper function which helps us to create dynamic few shots prompt
def create_examples(top_10,k):
  lst = []
  Prefix_prompt = "You are very good sentiment classifier.\nYour task is to predict tone of the sentence into two classes, positive or negative."
  for key in tqdm(top_10):
    if k == 0:
      exmpl = Prefix_prompt + "\n\n" + "Input: {}\nOutput:".format(key[0])
    else:
      Prefix_prompt_new = Prefix_prompt + "\nHere are some examples are given below for your reference.\n\n"
      examples = top_10[key][:k]
      str_ = ""
      for exp in examples:
        str_+= "Input: {}\nOutput: {}\n\n".format(exp['Input'],exp['Output'])

      exmpl = Prefix_prompt_new + str_ + "Input: {}\nOutput:".format(key[0])

    lst.append(exmpl)
  return lst


def create_examples_random(top_10,random_top_10,k):
  lst = []
  Prefix_prompt = "You are very good sentiment classifier.\nYour task is to predict tone of the sentence into two classes, positive or negative."
  for key in tqdm(top_10):
    if k == 0:
      exmpl = Prefix_prompt + "\n\n" + "Input: {}\nOutput:".format(key[0])
    else:
      Prefix_prompt_new = Prefix_prompt + "\nHere are some examples are given below for your reference.\n\n"
      examples = random_top_10[key[0]][:k]
      str_ = ""
      for exp in examples:
        str_+= "Input: {}\nOutput: {}\n\n".format(exp['Input'],exp['Output'])

      exmpl = Prefix_prompt_new + str_ + "\n" + "Input: {}\nOutput:".format(key[0])

    lst.append(exmpl)
  return lst

The Hugging Face pipeline module provides a high-level, easy-to-use API for performing various natural language processing tasks, such as sentiment analysis, text generation, and name entity recognition. Designed for simplicity, it allows users to apply cutting-edge transformer models like BERT and GPT with just a few lines of code, abstracting away the complexity involved in processing data and handling model inference.

**temperature**: In the context of text generation using models like GPT, the "temperature" parameter controls the randomness of token generation: a higher temperature increases diversity and unpredictability in the output, while a lower temperature makes the model's outputs more deterministic and conservative.

**top_p**: The "top_p" parameter in text generation, also known as nucleus sampling, restricts the model to choosing the smallest set of words whose cumulative probability exceeds the value of top_p, thereby balancing the trade-off between diversity and relevance in the generated text.

**repetition_penalty**: The "repetition_penalty" parameter in text generation models is used to discourage the model from repeating the same words or phrases, where a value greater than 1 decreases the likelihood of repetition, leading to more varied and creative outputs.

In [ ]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.1
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text2text-generation",
    model=model_flan,
    tokenizer=tokenizer_flan,
    generation_config=generation_config,
)

class_map = {"positive":0, "negative":1}
def get_results_for_20_samples(promt_list,seed_value,stop_flag,t_data):
    true = []
    pred = []
    for i,p in enumerate(tqdm(promt_list)):
        if i == stop_flag:
            break
        out = text_pipeline(p)
        try:
            pred.append(class_map[out[0]['generated_text']])
            true.append(test_data[i][-1])
        except:
            pass
    return true,pred

In [ ]:
random.shuffle(test_data)

keys = list(top_10.keys())
random.shuffle(keys)
shuffled_top_10 = {key: top_10[key] for key in keys}


keys = list(top_10_random.keys())

## Accuracy for each $k$ shot demonstrations

### When $k$=0, Zero Shot Learning

In [ ]:
k = 0

k_shots_F = create_examples(shuffled_top_10,k=k)

true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:22<03:05,  8.73it/s]


 Accuracy Faiss:
 0.5


## When $k$=1, 1-Shot Learning

In [ ]:
k = 1
k_shots_F = create_examples(shuffled_top_10,k=k)

print(k_shots_F[0])

100%|██████████| 1821/1821 [00:00<00:00, 112693.88it/s]

You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Too daft by half ... but supremely good natured .
Output: positive

Input: Pretty darn good , despite its smarty-pants aura .
Output:


In [ ]:
k = 1
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data)

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 348345.69it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: A simmering psychological drama in which the bursts of sudden violence are all the more startling for the slow buildup that has preceded them .
Output:
Sample Example Random:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: The film occasionally tries the viewer 's patience with slow pacing and a main character who sometimes defies sympathy , but it ultimately satisfies with its moving story .
Output: po

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:20<02:42,  9.96it/s]


 Accuracy Random:
 0.535

 Accuracy Faiss:
 0.53


## When $k$=2, 2-Shot Learning

In [ ]:
k = 2
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 282725.43it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: Though there are many tense scenes in Trapped , they prove more distressing than suspenseful .
Output: negative

Input: A simmering psychological drama in which the bursts of sudden violence are all the more startling for the slow buildup that has preceded them .
Output:
Sample Example Random:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: The film occasionally tries the viewer 's patience with slow pa

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:21<02:57,  9.15it/s]


 Accuracy Random:
 0.53

 Accuracy Faiss:
 0.535


## When $k$=*3*, 3-Shot Learning

In [ ]:
k = 3
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 160431.60it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: Though there are many tense scenes in Trapped , they prove more distressing than suspenseful .
Output: negative

Input: Not about scares but a mood in which an ominous , pervasive , and unknown threat lurks just below the proceedings and adds an almost constant mindset of suspense .
Output: positive

Input: A simmering psychological drama in which the bursts of sudden violence are all the more startling for the slow buildup that has preceded them .
Output:
Sample Example Random:
 You are very good sentiment classifier.
Your task is to predict to

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:23<03:13,  8.40it/s]


 Accuracy Random:
 0.535

 Accuracy Faiss:
 0.525


# When $k$=4, 4-Shot Learning

In [ ]:
k = 4
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 177401.11it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: Though there are many tense scenes in Trapped , they prove more distressing than suspenseful .
Output: negative

Input: Not about scares but a mood in which an ominous , pervasive , and unknown threat lurks just below the proceedings and adds an almost constant mindset of suspense .
Output: positive

Input: An old-fashioned scary movie , one that relies on lingering terror punctuated by sudden shocks and not constant bloodshed punctuated by flying guts .
Output: positive

Input: A simmering psychological drama in which the bursts of sudden viole

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:24<03:17,  8.19it/s]


 Accuracy Random:
 0.535

 Accuracy Faiss:
 0.53


## When $k$=5, 5-shot learning

In [ ]:
k = 5
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 183831.41it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: Though there are many tense scenes in Trapped , they prove more distressing than suspenseful .
Output: negative

Input: Not about scares but a mood in which an ominous , pervasive , and unknown threat lurks just below the proceedings and adds an almost constant mindset of suspense .
Output: positive

Input: An old-fashioned scary movie , one that relies on lingering terror punctuated by sudden shocks and not constant bloodshed punctuated by flying guts .
Output: positive

Input: Skillfully weaves both the elements of the plot and a powerfully ev

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:27<03:43,  7.24it/s]


 Accuracy Random:
 0.53

 Accuracy Faiss:
 0.53


In [ ]:
k = 6
k_shots_R = create_examples_random(shuffled_top_10,top_10_random,k=k)
k_shots_F = create_examples(shuffled_top_10,k=k)

# random.shuffle(k_shots)
# random.shuffle(test_data

print("Sample Example Faiss:\n", k_shots_F[0])
print("Sample Example Random:\n", k_shots_R[0])


true_f,pred_f = get_results_for_20_samples(k_shots_F,seed_value=123,stop_flag=200,t_data=test_data)
true_r,pred_r = get_results_for_20_samples(k_shots_R,seed_value=123,stop_flag=200,t_data=test_data)

print("\n Accuracy Random:\n", accuracy_score(true_r,pred_r))
print("\n Accuracy Faiss:\n", accuracy_score(true_f,pred_f))

100%|██████████| 1821/1821 [00:00<00:00, 45351.23it/s]


Sample Example Faiss:
 You are very good sentiment classifier.
Your task is to predict tone of the sentence into two classes, positive or negative.
Here are some examples are given below for your reference.

Input: Plunges you into a reality that is , more often then not , difficult and sad , and then , without sentimentalizing it or denying its brutality , transforms that reality into a lyrical and celebratory vision .
Output: positive

Input: Though there are many tense scenes in Trapped , they prove more distressing than suspenseful .
Output: negative

Input: Not about scares but a mood in which an ominous , pervasive , and unknown threat lurks just below the proceedings and adds an almost constant mindset of suspense .
Output: positive

Input: An old-fashioned scary movie , one that relies on lingering terror punctuated by sudden shocks and not constant bloodshed punctuated by flying guts .
Output: positive

Input: Skillfully weaves both the elements of the plot and a powerfully ev

  0%|          | 0/1821 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 11%|█         | 200/1821 [00:29<04:01,  6.72it/s]


 Accuracy Random:
 0.535

 Accuracy Faiss:
 0.535
